In [5]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent, PositionsTracker
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.ta.indicators import sma, ema
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.core.basics import Deal, Instrument, Order, Position, Signal

from qubx import QubxLogConfig
from qubx.backtester.simulator import simulate
from qubx.core.metrics import tearsheet
from qubx.trackers.sizers import FixedSizer, FixedRiskSizer

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Strat for signals testing

In [56]:
class StrategyForTracking(IStrategy):
    timeframe: str = "1Min"
    fast_period = 5
    slow_period = 12

    def on_fit(self, ctx: StrategyContext, fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f" -> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        signals = []
        for i in ctx.instruments:
            # logger.info(f"\t{i.symbol} : {ctx.broker_provider.get_quote(i.symbol)}")
            ohlc = ctx.ohlc(i, self.timeframe)
            fast = sma(ohlc.close, self.fast_period)
            slow = sma(ohlc.close, self.slow_period)
            pos = ctx.positions[i.symbol].quantity

            if pos <= 0:
                if (fast[0] > slow[0]) and (fast[1] < slow[1]):
                    # ctx.trade(i, abs(pos) + i.min_size * 10)
                    signals.append(i.signal(+1, stop=ohlc[1].low))

            if pos >= 0:
                if (fast[0] < slow[0]) and (fast[1] > slow[1]):
                    # ctx.trade(i, -pos - i.min_size * 10)
                    signals.append(i.signal(-1, stop=ohlc[1].high))

        return signals

    def ohlcs(self, timeframe: str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def tracker(self, ctx: StrategyContext) -> PositionsTracker:
        return PositionsTracker(FixedRiskSizer(5, 10_000, reinvest_profit=False))

In [57]:
r = CsvStorageDataReader("../tests/data/csv")

QubxLogConfig.set_log_level("DEBUG")

rep = simulate({
    
        "As Strategy": StrategyForTracking(
            timeframe="5Min", fast_period=5, slow_period=15),
    },
    r, 10000, ["BINANCE.UM:BTCUSDT"], 
    dict(type="ohlc", timeframe="1Min", nback=0),
    "5Min -1Sec",
    "vip0_usdt", 
    "2024-01-01", "2024-01-03", 
)

2024-07-30 16:33:39.647 [ ℹ️ ] StrategyForTracking new parameters:
	set timeframe <- 5Min
	set fast_period <- 5
	set slow_period <- 15


  0%|          | 0/1 [00:00<?, ?it/s]

2024-07-30 16:33:39.662 [ 🐞 ] 2024-01-01 00:00:00 Initiating simulated trading for binance.um for 10000 x 1.0 in USDT...
2024-07-30 16:33:39.662 [ ℹ️ ] SimulatedData.binance.um initialized
2024-07-30 16:33:39.662 [ 🐞 ] Triggering strategy on every 5Min bar after 4Min59S
2024-07-30 16:33:39.662 [ ℹ️ ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 0} for 
	['BTCUSDT'] 


  0%|          | 0/11524 [00:00<?, ?it/s]

2024-07-30 16:33:39.782 [ 🐞 ] [2024-01-01T00:00:00.001000000]: Invoking StrategyForTracking on_fit('2024-01-01 00:00:00', 'None')
2024-07-30 16:33:39.782 [ ℹ️ ]  -> Fit is called | fit_time: 2024-01-01 00:00:00 / prev: None
2024-07-30 16:33:39.782 [ ℹ️ ]  -> Fit is called | fit_time: 2024-01-01 00:00:00 / prev: None
2024-07-30 16:33:39.784 [ 🐞 ] [2024-01-01T00:00:00.001000000]: StrategyForTracking is fitted
2024-07-30 16:33:39.834 [ ℹ️ ] (StrategyContext) sending market sell for 0.234 of BTCUSDT ...
2024-07-30 16:33:39.836 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100001 MARKET SELL 0.234 executed at 42575.85
2024-07-30 16:33:39.836 [ ℹ️ ]   ::  traded -0.234 for BTCUSDT @ 42575.85 -> 0.00
2024-07-30 16:33:39.837 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100001 MARKET SELL 0.234 of BTCUSDT -> CLOSED
2024-07-30 16:33:39.837 [ 🐞 ] [SIM-ORDER-BTCUSDT-100001 / StrategyForTracking_BTCUSDT_17040672001] : MARKET SELL 0.234 of BTCUSDT  -> [CLOSED]
2024-07-30 16:33:39.839 [ 🐞 ] Executed -0.234 @ 42575.85 of

In [58]:
tearsheet(rep)

In [60]:
rep[0].executions_log

,instrument_id,side,filled_qty,price,commissions,commissions_quoted
timestamp,,,,,,
2024-01-01 02:19:59.999,BTCUSDT,sell,-0.234,42575.85,4.981374,USDT
2024-01-01 04:49:59.999,BTCUSDT,buy,0.469,42391.55,9.940818,USDT
2024-01-01 05:34:59.999,BTCUSDT,sell,-0.327,42344.05,6.923252,USDT
2024-01-01 06:24:59.999,BTCUSDT,buy,0.328,42346.95,6.944900,USDT
2024-01-01 11:34:59.999,BTCUSDT,sell,-0.243,42681.55,5.185808,USDT
2024-01-01 12:19:59.999,BTCUSDT,buy,0.241,42700.55,5.145416,USDT
2024-01-01 13:09:59.999,BTCUSDT,sell,-0.405,42727.45,8.652309,USDT
2024-01-01 13:24:59.999,BTCUSDT,buy,0.404,42754.55,8.636419,USDT
2024-01-01 14:09:59.999,BTCUSDT,sell,-0.338,42711.25,7.218201,USDT
